# Upload to Firebase!

Imports and setup

In [2]:
from time import sleep

import csv

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# if you don't have, go to Firebase Console > Project Settings > Service Accounts > Generate New Private Key
cred = credentials.Certificate('./talldle-game-firebase-adminsdk-fbsvc-94922110a3.json')

app = firebase_admin.initialize_app(cred)

db = firestore.client()

delete_collection_batch_size = 500

In [3]:
data_filename = './data/final_data.csv'
order_filename = './data/celeb-ordering-2025-12-23.csv'

Helpful functions

In [4]:
# from a snippet found on Firebase docs, https://firebase.google.com/docs/firestore/manage-data/delete-data#collections
def delete_collection(coll_ref, batch_size):
    if batch_size == 0:
        return

    docs = coll_ref.list_documents(page_size=batch_size)
    deleted = 0

    for doc in docs:
        #print(f"Deleting doc {doc.id} => {doc.get().to_dict()}")
        doc.delete()
        deleted = deleted + 1

    if deleted >= batch_size:
        return delete_collection(coll_ref, batch_size)

Celebrity data

Remove all the old data and upload all the new data

In [5]:
collection_ref = db.collection('data')

print("started deleting")
delete_collection(collection_ref, delete_collection_batch_size)
print("finished deleting")

sleep(10)

print("started uploading")
skip_first_row = True
with open(data_filename, 'r') as file:
    add_batch = db.batch()

    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        # convert keys to be lowercase
        row =  {k.lower(): v for k, v in row.items()}
        
        # convert height to be a float
        row['height'] = float(row['height'])
        
        add_batch.set(collection_ref.document(row['id']), row, merge=False)

add_batch.commit()
print("finished uploading")

started deleting
finished deleting
started uploading
finished uploading


Order data

Remove all the old data and upload all the new data

In [6]:
collection_ref = db.collection('order')

print("started deleting")
delete_collection(collection_ref, delete_collection_batch_size)
print("finished deleting")

sleep(10)

print("started uploading")
with open(order_filename, 'r') as file:
    add_batch = db.batch()

    day_index = 0

    csv_reader = csv.reader(file)
    for row in csv_reader:    
        add_batch.set(collection_ref.document(str(day_index)), {'data': row}, merge=False)

        day_index += 1

add_batch.commit()
print("finished uploading")

started deleting
finished deleting
started uploading
finished uploading
